# Select trials based on shape identifiers

In [10]:
import os
from glob import glob
import random
import pandas as pd 
import numpy as np
import shutil

from utils.io import load_log_trials, add_video_path

In [25]:
# Setting
data_dir = r"/media/yiting/NewVolume/Data/Videos"
annotation_dir = r"/home/yiting/Documents/Data/Videos/Annotations"
session_name = "2025-12-07"
log_dir = os.path.join(data_dir, session_name, "trial_logs")
log_trials = sorted(glob(os.path.join(log_dir, "*.json")))
logs = load_log_trials(log_trials)
video_folder_paths = sorted(glob(os.path.join(data_dir, session_name, "cameras", "*")))
logs = add_video_path(video_folder_paths, logs)
# Select shape IDs
# selected_shape_ids = ["G345", "G617", "G877", "G355", "B251", "G345", "G255", "G235", "G812", "B603", "G869"]
shape_ids = logs["shape_id"]
unique_shape_ids = list(set(shape_ids))
random.seed(42)
# selected_shape_ids = random.sample(unique_shape_ids, 11)
# print("Selected shape IDs:", selected_shape_ids)

## Copy selected videos to the annotation folder

In [26]:
for selected_shape_id in unique_shape_ids:
    shape_trials = []
    for idx, shape_id  in enumerate(shape_ids):
        if selected_shape_id in shape_id:
            shape_trials.append(idx)
    # Randomly select one trial from the shape_trials
    selected_trial_idx = random.choice(shape_trials)
    selected_video_folder = logs.iloc[selected_trial_idx]["video_folder_name"]
    video_folder_path = os.path.join(data_dir, session_name, "cameras", selected_video_folder)
    print(f"Shape ID: {selected_shape_id}, Trials: {shape_trials}, Selected Trial Index: {selected_trial_idx}, Video Folder Path: {video_folder_path}")
    # Copy the selected video folder to annotation directory
    os.makedirs(os.path.join(annotation_dir, session_name), exist_ok=True)
    shutil.copytree(video_folder_path, os.path.join(annotation_dir, session_name, selected_video_folder))

Shape ID: G226_0, Trials: [360, 361, 362, 363, 388, 393, 406, 407, 426, 430, 431], Selected Trial Index: 431, Video Folder Path: /media/yiting/NewVolume/Data/Videos/2025-12-07/cameras/2025-12-07_12-18-32_454057
Shape ID: G699_0, Trials: [86, 87, 109, 122, 129, 130, 131, 132, 158, 159], Selected Trial Index: 87, Video Folder Path: /media/yiting/NewVolume/Data/Videos/2025-12-07/cameras/2025-12-07_10-06-24_778105
Shape ID: G345_2, Trials: [88, 100, 108, 110, 111, 112, 143, 145, 152, 156, 167], Selected Trial Index: 88, Video Folder Path: /media/yiting/NewVolume/Data/Videos/2025-12-07/cameras/2025-12-07_10-06-36_589304
Shape ID: G750_2, Trials: [444, 451, 458, 459, 460, 468, 475, 478, 479, 495], Selected Trial Index: 460, Video Folder Path: /media/yiting/NewVolume/Data/Videos/2025-12-07/cameras/2025-12-07_12-25-50_648903
Shape ID: G358_0, Trials: [11, 12, 24, 25, 26, 27, 55, 62, 67, 79, 80], Selected Trial Index: 25, Video Folder Path: /media/yiting/NewVolume/Data/Videos/2025-12-07/cameras

## Concatenate videos during grasping window

In [27]:
from moviepy import VideoFileClip

cameras = ["camTo", "camTL", "camTR", "camBL", "camBR"]
trial_folders = glob(os.path.join(annotation_dir, session_name, "20*"))

for cam in cameras:
    output_dir = os.path.join(annotation_dir, session_name, f"{session_name}_concat")
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{cam}.mp4")
    temp_folder = os.path.join(annotation_dir, session_name, "temp_clips", cam)
    os.makedirs(temp_folder, exist_ok=True)
    temp_files_list = []
    for i, trial_folder in enumerate(trial_folders):
        video_path = os.path.join(trial_folder, f"{cam}.mp4")
        try:
            with VideoFileClip(video_path) as clip:
                # Extract the window from 2 second to 4 seconds
                # Note: .subclipped(start_time, end_time)
                cut_clip = clip.subclipped(2, 4)

                # Define temp filename
                temp_name = os.path.join(temp_folder, f"temp_{i:04d}.mp4")
                # Write immediately to disk to free memory
                # preset='ultrafast' speeds up the cut significantly
                cut_clip.write_videofile(
                    temp_name, 
                    codec="libx264", 
                    audio_codec="aac", 
                    preset="ultrafast", 
                    logger=None  # Turn off the progress bar spam
                )
                temp_files_list.append(f"file '{os.path.abspath(temp_name)}'")
        except Exception as e:
            print(f"Error processing {video_path}: {e}")
            continue

    # Use FFmpeg 'Demuxer' for the final merge (Zero RAM usage)
    # This is much faster than MoviePy's concatenate because it copies streams
    # instead of re-encoding pixels.
    list_file_path = "ffmpeg_list.txt"   
        # Write the file list formatted for FFmpeg
    with open(list_file_path, "w") as f:
        f.write("\n".join(temp_files_list))
        
    # Run FFmpeg command line
    # -f concat: Use the concat format
    # -safe 0: Allow unsafe file paths (absolute paths)
    # -c copy: COPY the streams (no re-encoding, takes seconds)
    os.system(f"ffmpeg -hide_banner -loglevel error -f concat -safe 0 -i {list_file_path} -c copy {output_path}")

    # 5. Cleanup temporary files (Optional)
    import shutil
    shutil.rmtree(temp_folder)
    os.remove(list_file_path)

    print(f"Done! Output saved to {output_path}")

Done! Output saved to /home/yiting/Documents/Data/Videos/Annotations/2025-12-07/2025-12-07_concat/camTo.mp4
Done! Output saved to /home/yiting/Documents/Data/Videos/Annotations/2025-12-07/2025-12-07_concat/camTL.mp4
Done! Output saved to /home/yiting/Documents/Data/Videos/Annotations/2025-12-07/2025-12-07_concat/camTR.mp4
Done! Output saved to /home/yiting/Documents/Data/Videos/Annotations/2025-12-07/2025-12-07_concat/camBL.mp4
Done! Output saved to /home/yiting/Documents/Data/Videos/Annotations/2025-12-07/2025-12-07_concat/camBR.mp4


## Create a shape-id list

In [16]:
# Setting
data_dir = r"/media/yiting/NewVolume/Data/Videos"
annotation_dir = r"/home/yiting/Documents/Data/Videos/Annotations"
session_dirs = glob(os.path.join(data_dir, "2025-12*"))
# sessions = ["2025-12-04", "2025-12-05"]
shape_ids_all = []
for session_dir in session_dirs:
    log_dir = os.path.join(session_dir, "trial_logs")
    log_trials = sorted(glob(os.path.join(log_dir, "*.json")))
    logs = load_log_trials(log_trials)
    shape_ids = logs["shape_id"]
    shape_ids_all.append(shape_ids)
if shape_ids_all:
    # This handles cases where sessions have different lengths (ragged arrays)
    flat_shape_ids = np.concatenate(shape_ids_all)
    unique_shape_ids = np.unique(flat_shape_ids)
else:
    unique_shape_ids = np.array([])

In [18]:
# Save the list of shape
save_path = os.path.join(annotation_dir, "shape_ids_2025-12.npy")
np.save(save_path, unique_shape_ids)